In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Hard spheres simulation (Lennard-Jones fluid)

In [3]:
from simtk import openmm as mm
from simtk import unit
from simtk.openmm import app
import numpy as np

In [ ]:
n_particles = 10
mass = 50.0 * unit.amu
sigma = 3.4 * unit.angstroms
epsilon = 0.238 * unit.kilocalories_per_mole
charge = 0.0 * unit.elementary_charge

In [ ]:
system = mm.System()

nb = mm.NonbondedForce()
nb.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
nb.setCutoffDistance(2.0*sigma)
nb.setUseDispersionCorrection(True) #

for particle_index in range(n_particles):
    system.addParticle(mass)
    nb.addParticle(charge, sigma, epsilon)

_ = system.addForce(nb)

In [ ]:
force = mm.CustomExternalForce('k * (z^2)')
force.addGlobalParameter('k', 100)
for particle_index in range(n_particles):
    force.addParticle(particle_index, [])

_ = system.addForce(force)

In [ ]:
box_x = 15.0 * unit.angstroms
box_y = 15.0 * unit.angstroms
box_z = 15.0 * unit.angstroms

v1 = [box_x, 0.0*unit.angstroms, 0.0*unit.angstroms]
v2 = [0.0*unit.angstroms, box_y, 0.0*unit.angstroms]
v3 = [0.0*unit.angstroms, 0.0*unit.angstroms, box_z]

system.setDefaultPeriodicBoxVectors(v1, v2, v3)

In [ ]:
coordinates = np.zeros([n_particles, 3]) * unit.angstroms

random_generator = np.random.default_rng()
coordinates[:,0] = random_generator.uniform(0.0, 1.0, n_particles) * box_x
coordinates[:,1] = random_generator.uniform(0.0, 1.0, n_particles) * box_y

In [ ]:
integrator = mm.LangevinIntegrator(300 * unit.kelvin, 1.0 / unit.picosecond, 0.002 * unit.picoseconds)

In [ ]:
platform = mm.Platform.getPlatformByName('CUDA')

In [ ]:
context = mm.Context(system, integrator, platform)

In [ ]:
context.setPositions(coordinates)
context.setVelocitiesToTemperature(300*unit.kelvin)

In [ ]:
state = context.getState(getEnergy=True)
state.getPotentialEnergy()

In [ ]:
for l in np.linspace(0.01, 1.0, num=100, endpoint=True):
    
    for ii in range(n_particles):
        nb.setParticleParameters(ii, charge, sigma*l, epsilon)
    
    mm.LocalEnergyMinimizer_minimize(context)
    integrator.step(1000)

In [ ]:
state = context.getState(getEnergy=True)
state.getPotentialEnergy()

In [ ]:
state = context.getState(getPositions=True)
coordinates = state.getPositions()
print(coordinates)

# Building a leaflet

In [ ]:
import openmembrane as om

In [ ]:
box_x = 2.7
box_y = 2.7
radii=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4]

In [ ]:
leaflet = om.build_leaflet(box_x, box_y, radii)

In [ ]:
leaflet

In [ ]:
fig, ax = plt.subplots()

ax.scatter(leaflet[:,0], leaflet[:,1], s=1)
ax.set_aspect('equal')
ax.set_xlim([0.0, box_x])
ax.set_ylim([0.0, box_y])

for ii in range(len(radii)):
    radius = radii[ii]
    x = leaflet[ii,0]
    y = leaflet[ii,1]
    new_circle = plt.Circle((x, y), radius, fill=False)
    ax.add_artist(new_circle)

plt.show()